In [1]:
import os
import json
import gc
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer
from typing import Dict, List, Set, Tuple, NamedTuple, Callable
import scipy
import scml
from scml import pandasx as pdx
from daigt.preprocess import en as pen
tim = scml.Timer()
tim.start()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
info = np.iinfo(np.int16)
print(f"int16, min={info.min}, max={info.max}")

int16, min=-32768, max=32767


In [2]:
text_col = "text_bsc"

In [3]:
df = pd.read_parquet("input/preprocess.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39515 entries, 0 to 39514
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   essay_id        39515 non-null  int64 
 1   generated       39515 non-null  int8  
 2   source          39515 non-null  object
 3   prompt          39515 non-null  object
 4   text            39515 non-null  object
 5   text_bsc        39515 non-null  object
 6   text_bow        39515 non-null  object
 7   text_bow_len    39515 non-null  int32 
 8   prompt_bsc      39515 non-null  object
 9   prompt_bow      39515 non-null  object
 10  prompt_bow_len  39515 non-null  int32 
dtypes: int32(2), int64(1), int8(1), object(7)
memory usage: 2.8+ MB


# Character level features

In [4]:
%%time
col = "ch_len"
df[col] = df[text_col].str.len()
df[col] = df[col].astype(np.int32)

def digit_frac(row) -> float:
    return pen.digit_frac(row[text_col])


def letter_frac(row) -> float:
    return pen.letter_frac(row[text_col])


def space_frac(row) -> float:
    return pen.space_frac(row[text_col])


def punc_frac(row) -> float:
    return pen.punc_frac(row[text_col])


def upper_frac(row) -> float:
    return pen.upper_frac(row[text_col])


def repeat_char_frac(row) -> float:
    return pen.repeat_char_frac(row[text_col])


def repeat_substring_frac(row) -> float:
    return pen.repeat_substring_frac(row[text_col])


char_fns: Dict[str, Callable] = {
    "ch_digit_frac": digit_frac,
    "ch_letter_frac": letter_frac,
    "ch_space_frac": space_frac,
    "ch_punc_frac": punc_frac,
    "ch_upper_frac": upper_frac,
    "ch_repeat_char_frac": repeat_char_frac,
    "ch_repeat_substring_frac": repeat_substring_frac,
}

for col, fn in char_fns.items():
    print(col)
    df[col] = df.progress_apply(fn, axis=1)
    df[col] = df[col].astype(np.float32)

ch_digit_frac


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39515/39515 [00:01<00:00, 22320.63it/s]


ch_letter_frac


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39515/39515 [00:01<00:00, 20269.40it/s]


ch_space_frac


...

ch_repeat_char_frac


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39515/39515 [00:05<00:00, 7412.32it/s]


ch_repeat_substring_frac


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39515/39515 [35:14<00:00, 18.69it/s]

CPU times: user 35min 22s, sys: 4.17 s, total: 35min 26s
Wall time: 35min 28s


# Review Data

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39515 entries, 0 to 39514
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   essay_id                  39515 non-null  int64  
 1   generated                 39515 non-null  int8   
 2   source                    39515 non-null  object 
 3   prompt                    39515 non-null  object 
 4   text                      39515 non-null  object 
 5   text_bsc                  39515 non-null  object 
 6   text_bow                  39515 non-null  object 
 7   text_bow_len              39515 non-null  int32  
 8   prompt_bsc                39515 non-null  object 
 9   prompt_bow                39515 non-null  object 
 10  prompt_bow_len            39515 non-null  int32  
 11  ch_len                    39515 non-null  int32  
 12  ch_digit_frac             39515 non-null  float32
 13  ch_letter_frac            39515 non-null  float32
 14  ch_spa

In [6]:
df.describe(percentiles=percentiles)

,essay_id,generated,text_bow_len,prompt_bow_len,ch_len,ch_digit_frac,ch_letter_frac,ch_space_frac,ch_punc_frac,ch_upper_frac,ch_repeat_char_frac,ch_repeat_substring_frac
count,39515.00000,39515.000000,39515.000000,39515.000000,39515.000000,39515.000000,39515.000000,39515.000000,39515.000000,39515.000000,39515.000000,39515.000000
mean,19757.00000,0.252891,2195.112818,87.917854,2229.710085,0.001559,0.799730,0.177670,0.021042,0.015673,0.017053,0.002284
std,11407.14228,0.434675,1006.480680,161.061500,1022.272265,0.003094,0.015733,0.013207,0.008840,0.016745,0.005114,0.008225
min,0.00000,0.000000,234.000000,2.000000,238.000000,0.000000,0.501969,0.005906,0.000000,0.000000,0.000890,0.000000
1%,395.14000,0.000000,773.000000,2.000000,784.140000,0.000000,0.759425,0.147226,0.007119,0.002213,0.007046,0.000000
5%,1975.70000,0.000000,951.000000,2.000000,964.000000,0.000000,0.773998,0.154341,0.010773,0.006222,0.009468,0.000000
10%,3951.40000,0.000000,1100.000000,2.000000,1116.000000,0.000000,0.780463,0.159455,0.012694,0.007421,0.010953,0.000000
20%,7902.80000,0.000000,1359.000000,2.000000,1380.000000,0.000000,0.787473,0.166591,0.015101,0.008875,0.012829,0.000000
30%,11854.20000,0.000000,1582.000000,2.000000,1606.000000,0.000000,0.792067,0.171273,0.016771,0.010243,0.014262,0.000000
40%,15805.60000,0.000000,1792.000000,2.000000,1822.000000,0.000000,0.795972,0.175124,0.018211,0.011763,0.015480,0.001138


In [7]:
%%time
df.to_parquet(f"output/features.parquet", index=False)
assert df.notna().all(axis=None)

CPU times: user 538 ms, sys: 47 ms, total: 585 ms
Wall time: 587 ms


In [8]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:35:29.821422
